### Product Categories Analysis
#### 1. Proportion of revenue levels per tag
#### 2. Mean, standard deviation, median dollar value per tag
#### 3. Transaction frequency per tag

To do:
1. Number of unique merchants/consumers per tag - Andrew
2. Age distribution by tag (add visualization) - Andrew
3. Consumer income distribution by tag (add visualization) - Andrew
4. Take rate * Dollar_Value - Andrew
5. Unique SA2 per merchant - Patrick
6. Analyze buying power (transaction frequency and monetary value) of SA2, possibly ranking weights to merchants by SA2 - Patrick
 6.1 Add SA2 (Income, Age and Buying Power) Geospatial Visualization - Nadya
7. Finalize features to use -> find correlation between features

### Analysis by Tag

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import geopandas as gpd
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

import geopandas as gpd
import folium
from pyspark.sql import SparkSession, Window, functions as F
from pyspark.sql.functions import countDistinct, col, date_format
import numpy as np
import pyspark.sql.functions as func
from pyspark.sql.types import (
    StringType,
    LongType,
    DoubleType,
    StructField,
    StructType,
    FloatType
)

import warnings
warnings.filterwarnings("ignore")

/Users/jasminchen/opt/anaconda3/envs/venv/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
# Start Spark Session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2 BNPL")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "8g")
    .getOrCreate()
)

22/09/15 14:15:13 WARN Utils: Your hostname, Jas-Mins-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.10.9 instead (on interface en0)
22/09/15 14:15:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/15 14:15:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# load BNPL dataset
consumer = spark.read.csv("../data/tables/tbl_consumer.csv", header=True, sep="|")
details = spark.read.parquet("../data/tables/consumer_user_details.parquet")
merchants = spark.read.parquet("../data/tables/tbl_merchants.parquet")

In [4]:
# load all transactions datasets
paths=['../data/tables/transactions_20210228_20210827_snapshot',
       '../data/tables/transactions_20210828_20220227_snapshot']

first = 1
for path in paths:
    if first:
        transactions = spark.read.parquet(path)
        print(f'added {path.split("/")[3]}')
        first = 0
    else:
        append_transactions = spark.read.parquet(path)
        transactions = transactions.union(append_transactions)
        print(f'added {path.split("/")[3]}')

added transactions_20210228_20210827_snapshot
added transactions_20210828_20220227_snapshot


In [5]:
age = gpd.read_file("../data/abs/sa2_age.gml")
income = gpd.read_file("../data/abs/sa2_income.gml")

DriverError: ../data/abs/sa2_age.gml: No such file or directory

In [ ]:
# load poa_to_sa2 dataset
poa_to_sa2 = spark.read.csv("../data/curated/poa_w_sa2.csv", header=True)

In [ ]:
consumer.show(1, vertical=True)

In [ ]:
details.show(1, vertical=True)

In [ ]:
transactions.agg({'order_datetime': 'max'}).show()
transactions.agg({'order_datetime': 'min'}).show()

In [ ]:
merchants.show(1, vertical=True)

In [ ]:
transactions.show(1, vertical=True)

In [ ]:
# rename columns
merchants = merchants.withColumnRenamed('name', 'merchant_name')
consumer = consumer.withColumnRenamed('name', 'consumer_name')

---
### Merge transaction with consumer and merchant details

In [ ]:
# Join consumers with their respective details
consumer_detail = consumer.join(details, on="consumer_id")

# Join consumers with their respective transactions
consumer_trx = consumer_detail.join(transactions, on="user_id")

# Join transactions with the respective merchants
df_trx = consumer_trx.join(merchants, on="merchant_abn")

In [ ]:
df_trx.show(5, vertical=True)

In [ ]:
df_trx.printSchema()

In [ ]:
poa_to_sa2.show(5, vertical=True)

In [ ]:
df_trx.show(5, vertical=True)

In [ ]:
# translate postcodes in transaction to sa2 codes
sa2_cols = ['poa_name_2016', 'sa2_maincode_2016', 'sa2_name_2016', 'geometry']
df_trx_sa2 = (df_trx \
                .join(poa_to_sa2[sa2_cols], 
                     on=[df_trx['postcode'] == poa_to_sa2['poa_name_2016']],
                     how='inner')
                .drop('poa_name_2016')
             )

In [ ]:
df_trx.show(5, vertical=True)

In [ ]:
# Explore product tags for purchase frequency
df_trx.groupby("tags").count().show(truncate=False)

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
@F.udf(StringType())
def normalize_tags(col):
    return col.replace("(", "[").replace(")", "]")[1:-1]

In [ ]:
df_trx_sa2 = df_trx_sa2.withColumn("tags", normalize_tags(F.col("tags")))

In [ ]:
df_trx.printSchema()

In [ ]:
df_trx_sa2 = df_trx_sa2.withColumn("categories", F.regexp_extract("tags", "(?<=\[)(.*?)(?=\])", 1))

In [ ]:
df_trx_sa2 = df_trx_sa2.withColumn("revenue_level", F.regexp_extract("tags", "(?<=,\s\[)([a-e]+?)(?=\],)", 1))

In [ ]:
df_trx_sa2 = df_trx_sa2.withColumn("take_rate", F.regexp_extract("tags", "(?<=\[take rate: )(.*?)(?=\])", 1))

In [ ]:
df_trx_sa2 = df_trx_sa2.withColumn("take_rate", F.col("take_rate").astype(FloatType()))

In [ ]:
@F.udf(StringType())
def clean_string(col):
    col = col.lower()
    return " ".join(col.split())

In [ ]:
df_trx_sa2 = df_trx_sa2.withColumn("categories", clean_string(F.col("categories")))

In [ ]:
category_take_rates = df_trx_sa2.select(["categories", "revenue_level", "take_rate"]).toPandas()

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [ ]:
import seaborn as sns
sns.boxplot(category_take_rates["revenue_level"], category_take_rates["take_rate"])
plt.show()

Revenue levels can be categorized by its take rate

In [ ]:
category_take_rates.groupby(["categories", "revenue_level"]).size().agg(
  {'count': lambda x: x, 'percent':lambda x: x / x.groupby(level=0).sum() * 100}
  ).unstack(level=0).reset_index()

We can see which categories provide the better amount in revenue for our BNPL company

In [ ]:
category_prices = df_trx_sa2.select(["categories", "dollar_value"]).toPandas()
category_prices = category_prices[category_prices["dollar_value"] >= 35]

In [ ]:
category_prices.groupby("categories").agg(["mean", "std", "median", "count"])

In [ ]:
sns.boxplot(category_prices[category_prices["categories"] == "telecom"].loc[:,"categories"], \
    category_prices[category_prices["categories"] == "telecom"].loc[:,"dollar_value"])
plt.show()

1. Number of unique merchants/consumers per tag - Andrew
2. Age distribution by tag (add visualization) - Andrew
3. Consumer income distribution by tag (add visualization) - Andrew
4. Take rate * Dollar_Value - Andrew

1. Number of Unique Merchants and Consumer per Tag

In [ ]:
df_trx_sa2.groupby("categories").agg(countDistinct("merchant_abn"), countDistinct("consumer_id"))

Calculating Revenue (Take Rate / 100 * Dollar Value) and taking the top 10 highest revenues

In [ ]:
@F.udf(FloatType())
def get_revenue(take_rate, dollar_value):
    return (take_rate / 100) * dollar_value

In [ ]:
trx_over35 = df_trx_sa2.where(F.col("dollar_value") >= 35)
trx_over35 = trx_over35.withColumn("revenue", get_revenue(F.col("take_rate"), F.col("dollar_value")))

In [ ]:
revenue_per_tag = trx_over35.groupby("categories").mean("revenue").sort("avg(revenue)", ascending=False)

In [ ]:
top10_tags = revenue_per_tag.select("categories").head(10)
for i in range(len(top10_tags)):
    top10_tags[i] = top10_tags[i].__getitem__('categories')

top10_tags

In [ ]:
revenue_per_tag.limit(10)

2. Age distribution by tag

In [ ]:
pd_age_tag = df_trx_sa2.select("sa2_maincode_2016", "categories").toPandas()

In [ ]:
pd_age_tag["sa2_maincode_2016"] = pd_age_tag["sa2_maincode_2016"].astype("float64")

In [ ]:
age_cols = ["sa2_main16", "persons_age_20_24", "persons_age_25_29" , "persons_age_30_34", "persons_age_35_39", "persons_age_40_44", \
    "persons_age_45_49", "persons_age_50_54", "persons_age_55_59", "persons_age_60_64", "persons_age_65_69", "persons_age_70_74", \
        "persons_age_75_79", "persons_age_80_84", "persons_age_85_plus"]

pd_age_tag = pd_age_tag.merge(age[age_cols], left_on="sa2_maincode_2016", right_on="sa2_main16", how="inner")

In [ ]:
pd_age_tag.head()

In [ ]:
pd_age_tag_final = pd_age_tag.groupby(["categories", "sa2_main16"]).mean().groupby("categories").sum()

In [ ]:
pd_age_tag_final["Total"] = pd_age_tag_final.sum(axis=1)

In [ ]:
for column in age_cols[1:]:
    pd_age_tag_final[column] = pd_age_tag_final[column] / pd_age_tag_final["Total"] * 100

In [ ]:
pd_age_tag_final.head()

In [ ]:
pd_age_tag_final = pd_age_tag_final.query("categories in @top10_tags")

In [ ]:
pd_age_tag_final.head(10)
# Values displayed is the percentage of people in that age bracket

3. Consumer income distribution by tag

In [ ]:
income_cols = ["sa2_code", "median_aud"]
pd_income_tag = pd_age_tag.merge(income[income_cols], left_on="sa2_maincode_2016", right_on="sa2_code")

In [ ]:
pd_income_tag_final = pd_income_tag.groupby(["categories", "sa2_main16"]).mean().groupby("categories").agg({'median_aud': 'mean'})

In [ ]:
pd_income_tag_final

In [ ]:
pd_income_tag_final = pd_income_tag_final.query("categories in @top10_tags")

In [ ]:
pd_income_tag_final

Median income for each tag is very similar

---
### 5. Unique SA2 per merchant


In [ ]:
# translate postcodes in transaction to sa2 codes
sa2_cols = ['poa_name_2016', 'sa2_maincode_2016', 'sa2_name_2016', 'geometry']
df_trx_sa2 = (df_trx \
                .join(poa_to_sa2[sa2_cols], 
                     on=[df_trx['postcode'] == poa_to_sa2['poa_name_2016']],
                     how='inner')
                .drop('poa_name_2016')
             )

In [ ]:
df_nunique_sa2 = (df_trx_sa2.groupby('merchant_abn')
                   .agg(countDistinct('sa2_maincode_2016')).toPandas())

In [ ]:
df_nunique_sa2.head()

In [ ]:
sns.displot(df_nunique_sa2["count(sa2_maincode_2016)"], 
            stat="density", kde=True, bins=25)
plt.title(f'Density Plot for "count(sa2_maincode_2016)"', fontsize=18)
plt.xlabel(f'count(sa2_maincode_2016)')
plt.show()

---
### 5. Unique consumer per merchant


In [ ]:
df_nunique_con = (df_trx_sa2.groupby('merchant_abn')
                   .agg(countDistinct('consumer_id')).toPandas())

In [ ]:
sns.displot(df_nunique_con["count(consumer_id)"], 
            stat="density", kde=True, bins=25)
plt.title(f'Density Plot for "count(consumer_id)"', fontsize=18)
plt.xlabel(f'count(consumer_id)')
plt.show()

---
### 6. Analyze buying power (transaction frequency and monetary value) of SA2, possibly ranking weights to merchants by SA2

In [ ]:
(df_trx_sa2.groupby(['sa2_maincode_2016'])
               .agg(func.mean('dollar_value'))
               .sort(['sa2_maincode_2016']))

In [ ]:
df_trx_sa2.printSchema()

In [ ]:
sa2_trx = (df_trx_sa2.groupby(['sa2_maincode_2016', 'state'])
             .agg({'order_id':'count', 'dollar_value':'sum'})
             .sort(['sa2_maincode_2016', 'state']))
unique_cons = (df_trx_sa2.groupby(['sa2_maincode_2016', 'state'])
               .agg(countDistinct('consumer_id'))
               .sort(['sa2_maincode_2016', 'state']))
unique_merc = (df_trx_sa2.groupby(['sa2_maincode_2016', 'state'])
               .agg(countDistinct('merchant_abn'))
               .sort(['sa2_maincode_2016', 'state']))
avg_trx_val = (df_trx_sa2.groupby(['sa2_maincode_2016', 'state'])
               .agg(func.mean('dollar_value'))
               .sort(['sa2_maincode_2016', 'state']))

def join_agg(sdf1, sdf2):
    '''
        take two dataframes and join the two dataframes
    '''
    sdf1 = (sdf1.alias("a") \
               .join(sdf2, 
                     on=['sa2_maincode_2016', 'state'], 
                     how='inner')
           )
    return sdf1
df_buy_pow = join_agg(sa2_trx, unique_cons)
df_buy_pow = join_agg(df_buy_pow, unique_merc)
df_buy_pow = join_agg(df_buy_pow, avg_trx_val)
    
# renaming a few columns
field_name_change = {"sum(dollar_value)": "total_dollar_value",
                     "avg(dollar_value)": "avg_dollar_value",
                     "count(order_id)": "transaction_freq",
                     "count(consumer_id)": "n_unique_consumer",
                     "count(merchant_abn)": "n_unique_merchant"}
for old, new in field_name_change.items():
    df_buy_pow = df_buy_pow.withColumnRenamed(old, new)

cols = ['sa2_maincode_2016', 'state', 'n_unique_consumer', 'transaction_freq', 
        'total_dollar_value', 'avg_dollar_value', 'n_unique_merchant']
df_buy_pow = df_buy_pow[cols].sort(['sa2_maincode_2016'])

df_buy_pow = (df_buy_pow.
             withColumn('spending_per_customer', 
                        col("total_dollar_value") / col("n_unique_consumer")))

df_buy_pow = (df_buy_pow.
             withColumn('num_trx_per_customer', 
                        col("transaction_freq") / col("n_unique_consumer")))

df_buy_pow = (df_buy_pow.
             withColumn('sales_per_merchant', 
                        col("total_dollar_value") / col("n_unique_merchant")))

df_buy_pow = (df_buy_pow.
             withColumn('num_trx_per_merchant', 
                        col("transaction_freq") / col("n_unique_merchant")))

In [ ]:
df_buy_pow.show(1, vertical=True)

In [ ]:
fields = ['n_unique_consumer', 'transaction_freq', 'total_dollar_value',
          'n_unique_merchant', 'spending_per_customer', 'num_trx_per_customer',
          'sales_per_merchant', 'num_trx_per_merchant', 'avg_dollar_value']

# get the distribution plot for the selected metrics
for field in fields:
    print(f'Distribution for {field}')
    df_field = df_buy_pow[[field]].toPandas()

    print(f'Minimum {field} by SA2: {df_field.min()[0]}')
    print(f'Median {field} by SA2: {df_field.median()[0]}')
    print(f'Maximum {field} by SA2: {df_field.max()[0]}')

    sns.displot(df_field[field], 
                stat="density", kde=True, bins=25)
    plt.title(f'Density Plot for {field}', fontsize=15)
    plt.xlabel(f'{field}')
    plt.show()
    
    print('---' * 25)

In [ ]:
# Rank SA2 with the highest buying power (spending per customer)
(df_buy_pow[['sa2_maincode_2016', 'state', 'spending_per_customer']]
            .sort('spending_per_customer', ascending=False).limit(10))

In [ ]:
# Rank SA2 with the highest average transaction value (avg_dollar_value)
(df_buy_pow[['sa2_maincode_2016', 'state', 'avg_dollar_value']]
            .sort('avg_dollar_value', ascending=False).limit(10))

In [ ]:
# Rank SA2 with the highest transaction frequency (num_trx_per_customer)
(df_buy_pow[['sa2_maincode_2016', 'state', 'num_trx_per_customer']]
            .sort('num_trx_per_customer', ascending=False).limit(10))

In [ ]:
# check if the table above by taking one SA2 code
(df_trx_sa2.filter(F.col('sa2_maincode_2016') == 309031239.0)
 .groupby('consumer_id').agg({
                               'order_id':'count',
                               'dollar_value': 'sum' 
                            }))

In [ ]:
age.head()

In [ ]:
# calculate population of male/female with age above 20
left_age = 25
right_age = 29
age['males_above_20'] = age['males_age_20_24']
age['females_above_20'] = age['females_age_20_24']
age['persons_above_20'] = age['persons_age_20_24']

while right_age < 89:
    if right_age == 89:
        right_age = 'plus'

    age['males_above_20'] += age[f'males_age_{left_age}_{right_age}']
    age['females_above_20'] += age[f'females_age_{left_age}_{right_age}']
    age['persons_above_20'] += age[f'persons_age_{left_age}_{right_age}']
    
    left_age += 5
    right_age += 5

In [ ]:
pdf_buy_pow = df_buy_pow.toPandas()

In [ ]:
# merge df_trx_sa2 with income, age, and population
income['sa2_code'] = income['sa2_code'].astype('float').astype('str')
income_col = ['sa2_code', 'median_aud', 'earners_persons', 
              'median_age_of_earners_years']
pdf_buy_pow = pdf_buy_pow.merge(income[income_col],
                                left_on='sa2_maincode_2016',
                                right_on='sa2_code', how='left')

age['sa2_main16'] = age['sa2_main16'].astype('float').astype('str')
age_col = ['sa2_main16', 'males_above_20', 
           'females_above_20', 'persons_above_20']
pdf_buy_pow = pdf_buy_pow.merge(age[age_col],
                                left_on='sa2_maincode_2016',
                                right_on='sa2_main16', how='left')

In [ ]:
pdf_buy_pow.info()

In [ ]:
# check correlation between features of interest
fields = ['n_unique_consumer', 'transaction_freq', 'total_dollar_value',
          'n_unique_merchant', 'spending_per_customer', 'num_trx_per_customer',
          'sales_per_merchant', 'num_trx_per_merchant', 'median_aud', 
          'earners_persons', 'median_age_of_earners_years', 
          'males_above_20', 'females_above_20', 'persons_above_20']
plt.figure(figsize=(12,7))
sns.heatmap(pdf_buy_pow[fields].corr(method='pearson'), 
            vmin=-1, vmax=1, annot=True, cmap='BrBG')

plt.title('Pearson Correlation Metrics', fontsize=18)
plt.show()

### Revenue by SA2

In [ ]:
df_trx_sa2 = df_trx_sa2.withColumn("revenue",
                                   get_revenue(F.col("take_rate"), 
                                               F.col("dollar_value")))

In [ ]:
cols = ["sa2_maincode_2016", "state", "revenue"]

revenue_sa2 = (df_trx_sa2[cols].groupby(["sa2_maincode_2016", "state"])
               .agg(func.mean("revenue"), func.sum("revenue")))

revenue_sa2.sort("avg(revenue)", ascending=False).limit(10)

---
### SA2 (Income, Age and Buying Power) Geospatial Visualization

In [ ]:
poa_to_sa2_pd = pd.read_csv("../data/curated/poa_w_sa2.csv")
poa_to_sa2_pd = poa_to_sa2_pd.dropna()

In [ ]:
from shapely import wkt

poa_to_sa2_pd['geometry'] = poa_to_sa2_pd['geometry'].astype('str').apply(wkt.loads)
gdf = gpd.GeoDataFrame(poa_to_sa2_pd, crs='epsg:4326')

In [ ]:
gdf['geometry'] = gdf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [ ]:
# create a JSON 
geoJSON = gdf[['sa2_maincode_2016', 'geometry']].drop_duplicates('sa2_maincode_2016').to_json()

#### 1. Income by SA2 Geospatial Visualization

In [ ]:
pd_income_tag.info()

In [ ]:
m = folium.Map(location=[-38.043995, 145.264296], tiles="Stamen Terrain", zoom_start=8)
m.add_child(folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    data=pd_income_tag, 
    columns = ['sa2_maincode_2016', 'median_aud'], 
    key_on = 'properties.sa2_maincode_2016', 
    fill_color='YlGnBu', 
    name='choropleth', 
    legend_name='Income by SA2 Area'))
m

#### 2. Age by SA2 Geospatial Visualization

In [ ]:
m = folium.Map(location=[-38.043995, 145.264296], tiles="Stamen Terrain", zoom_start=8)
m.add_child(folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    data=pd_age_tag, 
    columns = ['sa2_maincode_2016', 'persons_age_20_24'], 
    key_on = 'properties.sa2_maincode_2016', 
    fill_color='YlGnBu', 
    name='choropleth', 
    legend_name='Income by SA2 Area'))
m

In [ ]:
m = folium.Map(location=[-38.043995, 145.264296], tiles="Stamen Terrain", zoom_start=8)
m.add_child(folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    data=pd_age_tag, 
    columns = ['sa2_maincode_2016', 'persons_age_30_34'], 
    key_on = 'properties.sa2_maincode_2016', 
    fill_color='YlGnBu', 
    name='choropleth', 
    legend_name='Income by SA2 Area'))
m

In [ ]:
m = folium.Map(location=[-38.043995, 145.264296], tiles="Stamen Terrain", zoom_start=8)
m.add_child(folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    data=pd_age_tag, 
    columns = ['sa2_maincode_2016', 'persons_age_40_44'], 
    key_on = 'properties.sa2_maincode_2016', 
    fill_color='YlGnBu', 
    name='choropleth', 
    legend_name='Income by SA2 Area'))
m

#### 3. Buying Power by SA2 Geospatial Visualization
a. Number of Transaction per Customer

In [ ]:
m = folium.Map(location=[-38.043995, 145.264296], tiles="Stamen Terrain", zoom_start=8)
m.add_child(folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    data=pdf_buy_pow, 
    columns = ['sa2_maincode_2016', 'num_trx_per_customer'], 
    key_on = 'properties.sa2_maincode_2016', 
    fill_color='YlGnBu', 
    name='choropleth', 
    legend_name='Buying Power (Number of Transactions per Customer) by SA2'))
m

b. Spending per Customer

In [ ]:
m = folium.Map(location=[-38.043995, 145.264296], tiles="Stamen Terrain", zoom_start=8)
m.add_child(folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    data=pdf_buy_pow, 
    columns = ['sa2_maincode_2016', 'spending_per_customer'], 
    key_on = 'properties.sa2_maincode_2016', 
    fill_color='YlGnBu', 
    name='choropleth', 
    legend_name='Income by SA2 Area'))
m